In [2]:
from os import environ
from selenium.webdriver import Remote, ChromeOptions as Options
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection as Connection
from bs4 import BeautifulSoup

In [3]:
# AUTH = environ.get('AUTH', default='USER:PASS')
AUTH = "brd-customer-hl_94679ef3-zone-scraping_browser1:sflz39c9g431"

def scrape(url=None):
    if AUTH == 'USER:PASS':
        raise Exception('Provide Scraping Browsers credentials in AUTH ' +
                        'environment variable or update the script.')
    print('Connecting to Browser...')
    server_addr = f'https://{AUTH}@brd.superproxy.io:9515'
    connection = Connection(server_addr, 'goog', 'chrome')
    driver = Remote(connection, options=Options())
    try:
        print(f'Connected! Navigating to {url}...')
        driver.get(url)
        print('Navigated! Waiting captcha to detect and solve...')
        result = driver.execute('executeCdpCommand', {
            'cmd': 'Captcha.waitForSolve',
            'params': {'detectTimeout': 10 * 1000},
        })
        status = result['value']['status']
        print(f'Captcha status: {status}')
        data = driver.page_source
        return data
    finally:
        driver.quit()

In [4]:
url = "https://www.amazon.in/Apple-iPhone-13-128GB-Midnight/dp/B09G9HD6PD/ref=sr_1_1_sspa?crid=1KFEUPWN68RH&dib=eyJ2IjoiMSJ9.OCoJgZ8ghdguKvc7Ozmt3OyY3JIh2zh_9fXQPStOsFy5Gz9e2PbRJTLGIEpMmwnd9CVu_0AItnBKvK-Pq77gJAGlvlx3e9LCxkoBYj-AnFwlKgPYS58nL9_c7GNmqKuQcFPd0jyb08BNHDi_dh3FMtkU8OU0Qb9aEVfOPXgKt_O5U4yKqWrCID5IfTLted4P7JG6oZGx02NxuiqQj-Lk5-SUpHJeRXDa8_iFZw28XnTHLgxL3IvbOJPM4QFz5w0SXLMVp9Y3PZ-5JgAmjJSGouoKhAeLMUieTk_qpLjNdVI.mBsJGwDqlheBtYUa2AxA-YWkpGWLSF998HFKT3My270&dib_tag=se&keywords=iphone&qid=1728475253&s=electronics&sprefix=ihon%2Celectronics%2C354&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"
html = scrape(url=url)

Connecting to Browser...
Connected! Navigating to https://www.amazon.in/Apple-iPhone-13-128GB-Midnight/dp/B09G9HD6PD/ref=sr_1_1_sspa?crid=1KFEUPWN68RH&dib=eyJ2IjoiMSJ9.OCoJgZ8ghdguKvc7Ozmt3OyY3JIh2zh_9fXQPStOsFy5Gz9e2PbRJTLGIEpMmwnd9CVu_0AItnBKvK-Pq77gJAGlvlx3e9LCxkoBYj-AnFwlKgPYS58nL9_c7GNmqKuQcFPd0jyb08BNHDi_dh3FMtkU8OU0Qb9aEVfOPXgKt_O5U4yKqWrCID5IfTLted4P7JG6oZGx02NxuiqQj-Lk5-SUpHJeRXDa8_iFZw28XnTHLgxL3IvbOJPM4QFz5w0SXLMVp9Y3PZ-5JgAmjJSGouoKhAeLMUieTk_qpLjNdVI.mBsJGwDqlheBtYUa2AxA-YWkpGWLSF998HFKT3My270&dib_tag=se&keywords=iphone&qid=1728475253&s=electronics&sprefix=ihon%2Celectronics%2C354&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1...
Navigated! Waiting captcha to detect and solve...
Captcha status: solve_finished


In [5]:
# import setup
# setup.init_django(project_name="dcs")

In [6]:
soup = BeautifulSoup(html)

In [7]:
productTitle = soup.find('span', id='productTitle')
productTitleText = f"{productTitle.text}".strip()
productTitleText

'Apple iPhone 13 (128GB) - Midnight'

In [8]:
productPrice = soup.find_all('span', class_='a-price-whole')[0]
productPriceText = "".join([x for x in f"{productPrice.text}".strip() if x.isdigit() or x == '.'])
productPriceNum = float(productPriceText)
productPriceNum

42999.0

In [18]:
# product_data = soup.find('div', id='prodDetails')
# table = product_data.find('table')
# columns = [f"{x.text}".strip() for x in table.find_all('th')]

In [22]:
# table_data=[]
# for i, row in enumerate(table.find_all('tr')):
#     cells = row.find_all('td')
#     row_data = {columns[i]: f'{cell.text}'.strip() for cell in cells}
#     table_data.append(row_data)

In [24]:
# table_data

In [26]:
elements_with_attribute = soup.find_all(lambda tag: tag.has_attr('data-csa-c-asin'))
asins = [x.attrs.get('data-csa-c-asin') for x in elements_with_attribute if x]
asins = list(set([x for x in asins if x != ""]))
asins

['B09G9HD6PD']

In [30]:
# productDescription = soup.find('div', id='productDescription').text
# productDescription

In [28]:
featureBullets = soup.find('div', id='feature-bullets').text
featureBullets

' \n About this item    15 cm (6.1-inch) Super Retina XDR display    Cinematic mode adds shallow depth of field and shifts focus automatically in your videos    Advanced dual-camera system with 12MP Wide and Ultra Wide cameras; Photographic Styles, Smart HDR 4, Night mode, 4K Dolby Vision HDR recording    12MP TrueDepth front camera with Night mode, 4K Dolby Vision HDR recording    A15 Bionic chip for lightning-fast performance    \n ›  See more product details   '

In [32]:
average_rating = soup.find(id='averageCustomerReviews').find_all("span", class_='a-size-base')[0].text.strip()
average_rating = "".join([x for x in f"{average_rating}".strip() if x.isdigit() or x == '.'])
average_rating = float(average_rating)
average_rating

4.5

In [34]:
rating_data = soup.find(id='acrCustomerReviewText').text
rating_count = int(''.join([x for x in rating_data if x.isdigit()]))
rating_count

29850